In [38]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import torch.nn.functional as F

In [16]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', required=True, help='cifar10 | lsun | imagenet | folder | lfw | fake')
parser.add_argument('--dataroot', required=True, help='path to dataset')
parser.add_argument('--workers', type=int, help='number of data loading workers', default=2)
parser.add_argument('--batchSize', type=int, default=64, help='input batch size')
parser.add_argument('--imageSize', type=int, default=64, help='the height / width of the input image to network')
parser.add_argument('--nz', type=int, default=100, help='size of the latent z vector')
parser.add_argument('--ngf', type=int, default=64)
parser.add_argument('--ndf', type=int, default=64)
parser.add_argument('--niter', type=int, default=25, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.0002, help='learning rate, default=0.0002')
parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for adam. default=0.5')
parser.add_argument('--cuda', action='store_true', help='enables cuda')
parser.add_argument('--ngpu', type=int, default=1, help='number of GPUs to use')
parser.add_argument('--netG', default='', help="path to netG (to continue training)")
parser.add_argument('--netD', default='', help="path to netD (to continue training)")
parser.add_argument('--outf', default='.', help='folder to output images and model checkpoints')
parser.add_argument('--manualSeed', type=int, help='manual seed')

_StoreAction(option_strings=['--manualSeed'], dest='manualSeed', nargs=None, const=None, default=None, type=<type 'int'>, choices=None, help='manual seed', metavar=None)

In [17]:
args = {}

In [18]:
args['batch_size']=100
args['test_batch_size'] =1000

In [19]:
args['manualSeed'] = 5
print("Random Seed: ", args['manualSeed'])

Random Seed:  5


In [20]:
random.seed(args['manualSeed'])
torch.manual_seed(args['manualSeed'])

In [21]:
kwargs = {}
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['test_batch_size'], shuffle=True, **kwargs)

In [22]:
train_use = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args['batch_size'], shuffle=True, **kwargs)

In [23]:
def _calculate_fan_in_and_fan_out(tensor):
    dimensions = tensor.ndimension()
    if dimensions < 2:
        raise ValueError("Fan in and fan out can not be computed for tensor with less than 2 dimensions")

    if dimensions == 2:  # Linear
        fan_in = tensor.size(1)
        fan_out = tensor.size(0)
    else:
        num_input_fmaps = tensor.size(1)
        num_output_fmaps = tensor.size(0)
        receptive_field_size = 1
        if tensor.dim() > 2:
            receptive_field_size = tensor[0][0].numel()
        fan_in = num_input_fmaps * receptive_field_size
        fan_out = num_output_fmaps * receptive_field_size

    return fan_in, fan_out

In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
       # self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
      #  x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [25]:
neural_networks = {}

In [26]:
n_networks = 10
for i in range(n_networks):
    neural_networks[i] = Net()

In [27]:

training_loaders = {}

for i  in range(n_networks):

    training_loaders[i] = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args['batch_size'], shuffle=True, **kwargs)


In [28]:
import math

In [29]:
def weights_init(m, xavier=True, std_given=10., random_seed=1):
    classname = m.__class__.__name__
    random.seed(random_seed)
    torch.manual_seed(random_seed)
    print (classname)
    if classname.find('Conv') != -1:
       # m.weight.data.normal_(0.0, 0.02)
        
        if xavier:
            z = _calculate_fan_in_and_fan_out(m.weight.data)
            std = math.sqrt(2.0 / (z[0] + z[1]))
            m.weight.data.uniform_(-std, std)
        else:
            m.weight.data.uniform_(-std_given, std_given)
    if classname.find('Linear') != -1:
        if xavier:
            z = _calculate_fan_in_and_fan_out(m.weight.data)
            std = math.sqrt(2.0 / (z[0] + z[1]))
            m.weight.data.uniform_(-std, std)
            
            m.bias.data.fill_(0.0)
        else:
            m.weight.data.uniform_(-std_given, std_given)
            m.bias.data.uniform_(-std_given, std_given)



In [30]:
for i in range(1, n_networks - 1):
    magnitudes = 10 ** (- i + 4)
    g = lambda x: weights_init(x, xavier=False, std_given=magnitudes, random_seed=i)
    neural_networks[i].apply(g)

Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net
Conv2d
Conv2d
Linear
Linear
Net


In [31]:
neural_networks[0].apply(weights_init)

Conv2d
Conv2d
Linear
Linear
Net


Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [32]:
keep = [0, 1, 2, 3, 4, 5, 9]

In [33]:
args['lr'] = 0.01
args['momentum'] = 0.0

In [34]:
args['cuda'] = False

In [35]:
optimizers = {}

for i in neural_networks:
    optimizers[i] = optim.SGD(neural_networks[i].parameters(), lr=args['lr'], momentum=args['momentum'])

In [44]:
def train(epoch, model, optimizer, train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
#         if batch_idx % 10 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.data[0]))

    model.eval()
    train_loss = 0
    correct = 0
    for data, target in train_use:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        train_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
    print (train_loss, train_use.dataset)
    train_loss /= len(train_use.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
    return 100. * correct / len(train_loader.dataset)

In [36]:
results = {}
for i in range(n_networks):
    results[i] = []

In [40]:
first_results = {}
for i in keep:
    model = neural_networks[i]
    model.eval()
    train_loss = 0
    correct = 0
    for data, target in train_use:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        train_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
    print (train_loss, train_use.dataset)
    train_loss /= len(train_use.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss, correct, len(train_use.dataset),
        100. * correct / len(train_use.dataset)))
    first_results[i] = 100. * correct / len(train_use.dataset)

138891.348297 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 2.3149, Accuracy: 3779/60000 (6%)

1.06181579186e+20 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 1769692986428987.7500, Accuracy: 3682/60000 (6%)

5.51512738156e+15 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 91918789692.6208, Accuracy: 6695/60000 (11%)

1.44739468902e+12 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 24123244.8171, Accuracy: 5880/60000 (10%)

90136999.3984 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 1502.2833, Accuracy: 8226/60000 (14%)

138549.15387 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 2.3092, Accuracy: 4290/60000 (7%)

138507.113861 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 2.3085, Accuracy: 5914/60000 (10%)



In [41]:
first_results

{0: 6.298333333333333,
 1: 6.136666666666667,
 2: 11.158333333333333,
 3: 9.8,
 4: 13.71,
 5: 7.15,
 9: 9.856666666666667}

In [42]:
for i in keep:
    results[i].append(first_results[i])

In [ ]:
for epoch in range(1, 20 + 1):
    for i in keep:
        res = train(epoch, neural_networks[i], optimizers[i], training_loaders[i])
        results[i].append(res)

20324.3091087 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 0.3387, Accuracy: 53827/60000 (90%)

nan <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: nan, Accuracy: 5923/60000 (10%)

nan <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: nan, Accuracy: 5923/60000 (10%)

410316.272827 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 6.8386, Accuracy: 5918/60000 (10%)

138601.473007 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 2.3100, Accuracy: 6143/60000 (10%)

18743.7354488 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 0.3124, Accuracy: 54556/60000 (91%)

16628.5409222 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 0.2771, Accuracy: 55000/60000 (92%)

13551.8254471 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Tes

3539.02838963 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 0.0590, Accuracy: 58914/60000 (98%)

nan <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: nan, Accuracy: 5923/60000 (10%)

nan <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: nan, Accuracy: 5923/60000 (10%)

153014.941299 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 2.5502, Accuracy: 6742/60000 (11%)

138052.343124 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 2.3009, Accuracy: 6753/60000 (11%)

3808.01870185 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 0.0635, Accuracy: 58836/60000 (98%)

3246.66483688 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Test set: Average loss: 0.0541, Accuracy: 59005/60000 (98%)

3677.92500833 <torchvision.datasets.mnist.MNIST object at 0x7f2f894f5fd0>

Tes

In [46]:
1

1

In [484]:
results

{0: [8.068333333333333,
  91.935,
  95.08,
  96.09166666666667,
  97.08666666666667,
  97.42833333333333,
  97.73833333333333,
  97.81666666666666,
  98.01,
  98.32666666666667,
  98.36166666666666,
  98.40333333333334,
  98.56,
  98.675,
  98.72166666666666,
  98.75833333333334,
  98.83833333333334,
  98.82,
  98.84666666666666,
  98.85333333333334,
  99.03166666666667],
 1: [6.136666666666667,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666],
 2: [11.153333333333334,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.871666666666666,
  9.8716

In [462]:
import ujson

In [442]:
with open('sgd_paths_cnn', 'w') as f:
    ujson.dump(results, f)


In [463]:
with open('sgd_paths_cnn_2', 'w') as f:
    ujson.dump(results, f)
